# Trend Following Strategy
This notebook implements a trend-following strategy using EMA, MACD, ADX, and ATR filters.
It includes:
- Entry/exit logic with ADX and EMA filters
- ATR-based SL/TP
- Optimization setup
- Performance evaluation

In [5]:
import pandas as pd
import numpy as np
from backtesting import Backtest, Strategy
from backtesting.test import GOOG

class AppliedStrategy(Strategy):
    # Indicator parameters
    ema_short_period = 20
    ema_long_period = 50
    macd_fast = 12
    macd_slow = 26
    macd_signal = 9
    adx_period = 14
    atr_period = 14

    # Risk management
    atr_mult_sl = 2     # SL = 2 * ATR
    atr_mult_tp = 3.5   # TP = 3.5 * ATR
    adx_threshold = 20

    def init(self):
        df = self.data.df
        close = df['Close']
        high = df['High']
        low = df['Low']

        # EMA Indicators
        ema_short = close.ewm(span=self.ema_short_period, adjust=False).mean()
        ema_long = close.ewm(span=self.ema_long_period, adjust=False).mean()

        # MACD Indicators
        macd_line = close.ewm(span=self.macd_fast, adjust=False).mean() - close.ewm(span=self.macd_slow, adjust=False).mean()
        signal_line = macd_line.ewm(span=self.macd_signal, adjust=False).mean()

        # Register indicators
        self.ema_short = self.I(lambda: ema_short)
        self.ema_long = self.I(lambda: ema_long)
        self.macd_line = self.I(lambda: macd_line)
        self.signal_line = self.I(lambda: signal_line)
        self.adx = self.I(self.calculate_adx, high, low, close, self.adx_period)
        self.atr = self.I(self.calculate_atr, high, low, close, self.atr_period)

    def calculate_atr(self, high, low, close, period):
        tr1 = high - low
        tr2 = abs(high - close.shift())
        tr3 = abs(low - close.shift())
        tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
        return tr.rolling(period).mean()

    def calculate_adx(self, high, low, close, period):
        plus_dm = high.diff()
        minus_dm = -low.diff()
        plus_dm[plus_dm < 0] = 0
        minus_dm[minus_dm < 0] = 0

        tr1 = high - low
        tr2 = abs(high - close.shift())
        tr3 = abs(low - close.shift())
        tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
        atr = tr.rolling(period).mean()

        plus_di = 100 * (plus_dm.rolling(period).sum() / atr)
        minus_di = 100 * (minus_dm.rolling(period).sum() / atr)
        dx = 100 * abs(plus_di - minus_di) / (plus_di + minus_di)
        adx = dx.rolling(period).mean()
        return adx

    def next(self):
        price = self.data.Close[-1]
        ema_s = self.ema_short[-1]
        ema_l = self.ema_long[-1]
        macd = self.macd_line[-1]
        signal = self.signal_line[-1]
        adx_val = self.adx[-1]
        atr_val = self.atr[-1]

        if not self.position:
            if ema_s > ema_l and macd > signal and adx_val > self.adx_threshold:
                sl = price - self.atr_mult_sl * atr_val
                tp = price + self.atr_mult_tp * atr_val
                self.buy(sl=sl, tp=tp)

        elif ema_s < ema_l or macd < signal or adx_val < self.adx_threshold:
            self.position.close()

# Run the backtest
bt = Backtest(GOOG, AppliedStrategy, cash=100_000, commission=0.002, exclusive_orders=True)
stats = bt.run()
bt.plot()
print(stats)



Backtest.run:   0%|          | 0/2120 [00:00<?, ?bar/s]

Start                     2004-08-19 00:00:00
End                       2013-03-01 00:00:00
Duration                   3116 days 00:00:00
Exposure Time [%]                    31.56425
Equity Final [$]                 201966.33297
Equity Peak [$]                  216933.34204
Commissions [$]                   64941.37417
Return [%]                          101.96633
Buy & Hold Return [%]               535.49582
Return (Ann.) [%]                     8.59626
Volatility (Ann.) [%]                17.24982
CAGR [%]                               5.8495
Sharpe Ratio                          0.49834
Sortino Ratio                         0.85048
Calmar Ratio                            0.385
Alpha [%]                            -9.96247
Beta                                  0.20902
Max. Drawdown [%]                   -22.32815
Avg. Drawdown [%]                    -5.01233
Max. Drawdown Duration     1942 days 00:00:00
Avg. Drawdown Duration      145 days 00:00:00
# Trades                          